In [ ]:
pip install lightgbm xgboost optuna tensorflow

In [37]:
#encoding=utf8
from sklearn.model_selection import KFold, StratifiedKFold
import pandas as pd
import numpy as np
from scipy import sparse
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import math

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor, HistGradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss,mean_absolute_error,mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler,Normalizer,StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB

In [2]:
def stacking(clf, train_x, train_y, test_x, clf_name, class_num = 3):
    train = np.zeros((train_x.shape[0], class_num))
    test = np.zeros((test_x.shape[0], class_num))
    test_pre = np.empty((folds, test_x.shape[0], class_num))
    cv_scores=[]
    for i, (train_index, test_index) in enumerate(kf):
        tr_x = train_x[train_index];tr_y = train_y[train_index]
        te_x = train_x[test_index]; te_y = train_y[test_index]
        
        if clf_name in ["rf","ada","gb","et","lr","knn","mnb","ovr","gnb"]:
            clf.fit(tr_x,tr_y)
            pre = clf.predict_proba(te_x)
            train[test_index] = pre
            test_pre[i,:] = clf.predict_proba(test_x)
            cv_scores.append(log_loss(te_y, pre))
        elif clf_name in ["lsvc"]:
            clf.fit(tr_x,tr_y)
            pre = clf.decision_function(te_x)
            train[test_index] = pre
            test_pre[i,:] = clf.decision_function(test_x)
            cv_scores.append(log_loss(te_y, pre))
        elif clf_name in ["xgb"]:
            train_matrix = clf.DMatrix(tr_x, label = tr_y, missing = -1)
            test_matrix = clf.DMatrix(te_x, label = te_y, missing = -1)
            z = clf.DMatrix(test_x, label = te_y, missing = -1)
            params = {'booster': 'gbtree',
                      'objective': 'multi:softprob',
                      'eval_metric': 'mlogloss',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.03,
                      'tree_method': 'exact',
                      'seed': 2017,
                      'nthread': 12,
                      "num_class": class_num}
            num_round = 10000
            early_stopping_rounds = 100
            watchlist = [(train_matrix, 'train'),
                         (test_matrix, 'eval')
                         ]
            if test_matrix:
                model = clf.train(params, train_matrix, num_boost_round = num_round, evals = watchlist, \
                                  early_stopping_rounds = early_stopping_rounds)
                pre = model.predict(test_matrix, ntree_limit = model.best_ntree_limit)
                train[test_index] = pre
                test_pre[i, :] = model.predict(z, ntree_limit = model.best_ntree_limit)
                cv_scores.append(log_loss(te_y, pre))
        elif clf_name in ["lgb"]:
            train_matrix = clf.Dataset(tr_x, label = tr_y)
            test_matrix = clf.Dataset(te_x, label = te_y)
            #z = clf.Dataset(test_x, label=te_y)
            #z=test_x
            params = {
                      'boosting_type': 'gbdt',
                      'objective': 'multiclass',
                      'metric': 'multi_logloss',
                      'min_child_weight': 1.5,
                      'num_leaves': 2**5,
                      'lambda_l2': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'learning_rate': 0.03,
                      'tree_method': 'exact',
                      'seed': 2017,
                      'nthread': 12,
                      "num_class": class_num,
                      'silent': True}
            num_round = 10000
            early_stopping_rounds = 100
            if test_matrix:
                model = clf.train(params, train_matrix, num_round,valid_sets = test_matrix, \
                                  early_stopping_rounds = early_stopping_rounds)
                pre = model.predict(te_x, num_iteration = model.best_iteration)
                train[test_index] = pre
                test_pre[i, :] = model.predict(test_x, num_iteration = model.best_iteration)
                cv_scores.append(log_loss(te_y, pre))
        elif clf_name in ["nn"]:
            from keras.layers import Dense, Dropout, BatchNormalization,SReLU
            from keras.optimizers import SGD,RMSprop
            from keras.callbacks import EarlyStopping, ReduceLROnPlateau
            from keras.utils import np_utils
            from keras.regularizers import l2
            from keras.models import Sequential
            clf = Sequential()
            clf.add(Dense(64, input_dim = tr_x.shape[1], activation = "relu", W_regularizer = l2()))
            #clf.add(SReLU())
            #clf.add(Dropout(0.2))
            clf.add(Dense(64,activation = "relu", W_regularizer = l2()))
            #clf.add(SReLU())
            #clf.add(Dense(64, activation="relu", W_regularizer=l2()))
            # model.add(Dropout(0.2))
            clf.add(Dense(class_num, activation = "softmax"))
            clf.summary()
            early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20)
            reduce = ReduceLROnPlateau(min_lr = 0.0002, factor = 0.05)
            clf.compile(optimizer="rmsprop", loss="categorical_crossentropy")
            clf.fit(tr_x, tr_y, batch_size = 640, nb_epoch = 1000, validation_data = [te_x, te_y], callbacks = [early_stopping, reduce])
            pre = clf.predict_proba(te_x)
            train[test_index] = pre
            test_pre[i,:] = clf.predict_proba(test_x)
            cv_scores.append(log_loss(te_y, pre))
        else:
            raise IOError("Please add new clf.")
        print("%s now score is:"%clf_name,cv_scores)
        with open("score.txt","a") as f:
            f.write("%s now score is:"%clf_name+str(cv_scores)+"\n")
    test[:]=test_pre.mean(axis=0)
    print("%s_score_list:"%clf_name,cv_scores)
    print("%s_score_mean:"%clf_name,np.mean(cv_scores))
    with open("score.txt", "a") as f:
        f.write("%s_score_mean:"%clf_name+str(np.mean(cv_scores))+"\n")
    return train.reshape(-1,class_num),test.reshape(-1,class_num)

def rf(x_train, y_train, x_valid):
    randomforest = RandomForestClassifier(n_estimators=1200, max_depth=20, n_jobs=-1, random_state=2017, max_features="auto",verbose=1)
    rf_train, rf_test = stacking(randomforest, x_train, y_train, x_valid,"rf")
    return rf_train, rf_test,"rf"

def ada(x_train, y_train, x_valid):
    adaboost = AdaBoostClassifier(n_estimators=50, random_state=2017, learning_rate=0.01)
    ada_train, ada_test = stacking(adaboost, x_train, y_train, x_valid,"ada")
    return ada_train, ada_test,"ada"

def gb(x_train, y_train, x_valid):
    gbdt = GradientBoostingClassifier(learning_rate=0.04, n_estimators=100, subsample=0.8, random_state=2017,max_depth=5,verbose=1)
    gbdt_train, gbdt_test = stacking(gbdt, x_train, y_train, x_valid,"gb")
    return gbdt_train, gbdt_test,"gb"

def et(x_train, y_train, x_valid):
    extratree = ExtraTreesClassifier(n_estimators=1200, max_depth=35, max_features="auto", n_jobs=-1, random_state=2017,verbose=1)
    et_train, et_test = stacking(extratree, x_train, y_train, x_valid,"et")
    return et_train, et_test,"et"

def ovr(x_train, y_train, x_valid):
    est=RandomForestClassifier(n_estimators=400, max_depth=16, n_jobs=-1, random_state=2017, max_features="auto",
                               verbose=1)
    ovr = OneVsRestClassifier(est,n_jobs=-1)
    ovr_train, ovr_test = stacking(ovr, x_train, y_train, x_valid,"ovr")
    return ovr_train, ovr_test,"ovr"

def xgb(x_train, y_train, x_valid):
    xgb_train, xgb_test = stacking(xgboost, x_train, y_train, x_valid,"xgb")
    return xgb_train, xgb_test,"xgb"

def lgb(x_train, y_train, x_valid):
    xgb_train, xgb_test = stacking(lightgbm, x_train, y_train, x_valid,"lgb")
    return xgb_train, xgb_test,"lgb"

def gnb(x_train, y_train, x_valid):
    gnb=GaussianNB()
    gnb_train, gnb_test = stacking(gnb, x_train, y_train, x_valid,"gnb")
    return gnb_train, gnb_test,"gnb"

def lr(x_train, y_train, x_valid):
    logisticregression=LogisticRegression(n_jobs=-1,random_state=2017,C=0.1,max_iter=200)
    lr_train, lr_test = stacking(logisticregression, x_train, y_train, x_valid, "lr")
    return lr_train, lr_test, "lr"

def fm(x_train, y_train, x_valid):
    pass


def lsvc(x_train, y_train, x_valid):

    #linearsvc=SVC(probability=True,kernel="linear",random_state=2017,verbose=1)
    #linearsvc=SVC(probability=True,kernel="linear",random_state=2017,verbose=1)
    linearsvc=LinearSVC(random_state=2017)
    lsvc_train, lsvc_test = stacking(linearsvc, x_train, y_train, x_valid, "lsvc")
    return lsvc_train, lsvc_test, "lsvc"

def knn(x_train, y_train, x_valid):
    #pca = PCA(n_components=10)
    #pca.fit(x_train)
    #x_train = pca.transform(x_train)
    #x_valid = pca.transform(x_valid)

    kneighbors=KNeighborsClassifier(n_neighbors=200,n_jobs=-1)
    knn_train, knn_test = stacking(kneighbors, x_train, y_train, x_valid, "knn")
    return knn_train, knn_test, "knn"

def nn(x_train, y_train, x_valid):
    from keras.utils import np_utils
    y_train = np_utils.to_categorical(y_train)
    nn_train, nn_test = stacking("", x_train, y_train, x_valid, "nn")
    return nn_train, nn_test, "nn"

In [10]:
## Reading the data
train = pd.read_csv('Data/W23P2_train.csv')
test = pd.read_csv('Data/W23P2_test.csv')
sub = pd.read_csv('Data/Sample_Submission.csv')

In [11]:
## Transforming interest_level label to numeric
train['interest_level'] = np.where(train['interest_level'] == 'low', 0, 
                                   np.where(train['interest_level'] == 'medium', 1, 2))

## Removing training observations with prices higher than $40,000
train = train[train['price'] < 40000].reset_index(drop = True)

## Removing outlier locations
train = train[(train['latitude'] < 43) & (train['latitude'] > 1)].reset_index(drop = True)

In [12]:
## Combining data frames for cleaning and feature engineering purposes
train.insert(0, 'train', 1)
test.insert(0, 'train', 0)
full_data = pd.concat([train.drop(columns = ['interest_level']), test.drop(columns = ['ID'])])

## Making sure address variable is a string
full_data['street_address'] = full_data['street_address'].astype(str)

## Altering the missing values
full_data['street_address'] = np.where(full_data['street_address'] == 'nan', '123 nan', full_data['street_address'])

## Changing street address variable to all lowercase
full_data['street_address'] = full_data['street_address'].str.lower() 

## Splitting street_address into number and street
addies = pd.DataFrame(list(full_data['street_address'].str.split(' ', 1)), columns = ['number', 'street'])
addies['street'] = ' ' + addies['street'] + ' '

## Cleaning street variable
addies['street'] = addies['street'].str.replace(' st ', ' street', regex = True)
addies['street'] = addies['street'].str.replace(' st. ', ' street', regex = True)
addies['street'] = addies['street'].str.replace(' blvd ', ' boulevard ', regex = True)
addies['street'] = addies['street'].str.replace(' ave ', ' avenue ', regex = True)
addies['street'] = addies['street'].str.replace(' ave. ', ' avenue ', regex = True)
addies['street'] = addies['street'].str.replace(' dr ', ' drive ', regex = True)
addies['street'] = addies['street'].str.replace(' pl ', ' place ', regex = True)
addies['street'] = addies['street'].str.replace(' e ', ' east ', regex = True)
addies['street'] = addies['street'].str.replace(' w ', ' west ', regex = True)
addies['street'] = addies['street'].str.replace(' first ', ' 1st ', regex = True)
addies['street'] = addies['street'].str.replace(' second ', ' 2nd ', regex = True)
addies['street'] = addies['street'].str.replace(' third ', ' 3rd ', regex = True)
addies['street'] = addies['street'].str.replace(' fourth ', ' 4th ', regex = True)
addies['street'] = addies['street'].str.replace(' fifth ', ' 5th ', regex = True)
addies['street'] = addies['street'].str.replace(' sixth ', ' 6th ', regex = True)
addies['street'] = addies['street'].str.replace(' seventh ', ' 7th ', regex = True)
addies['street'] = addies['street'].str.replace(' eighth ', ' 8th ', regex = True)
addies['street'] = addies['street'].str.replace(' ninth ', ' 9th ', regex = True)

addies['street'] = addies['street'].str.replace('1 ', '1st ', regex = True)
addies['street'] = addies['street'].str.replace('2 ', '2nd ', regex = True)
addies['street'] = addies['street'].str.replace('3 ', '3rd ', regex = True)
addies['street'] = addies['street'].str.replace('4 ', '4th ', regex = True)
addies['street'] = addies['street'].str.replace('5 ', '5th ', regex = True)
addies['street'] = addies['street'].str.replace('6 ', '6th ', regex = True)
addies['street'] = addies['street'].str.replace('7 ', '7th ', regex = True)
addies['street'] = addies['street'].str.replace('8 ', '8th ', regex = True)
addies['street'] = addies['street'].str.replace('9 ', '9th ', regex = True)


## Adding cleaned street_address variable back into the full_data data-frame
full_data['street_address'] = addies['number'] + ' ' + addies['street']

/tmp/ipykernel_24027/329181946.py:16: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  addies = pd.DataFrame(list(full_data['street_address'].str.split(' ', 1)), columns = ['number', 'street'])


In [14]:
## Adding geolocation features
full_data['geo_area_50'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:(int(x[0] * 50) % 50) * 50 + (int(-x[1] * 50) % 50), axis = 1)                                         
                         
full_data['geo_area_100'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:(int(x[0] * 100) % 100) * 100 + (int(-x[1] * 100) % 100), axis = 1)                                         
  
full_data['geo_area_200'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:(int(x[0] * 200) % 200) * 200 + (int(-x[1] * 200) % 200), axis = 1)                                         

## Financial district
lat = 40.705628
lon = -74.010278
full_data['distance_to_fin'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:math.sqrt((x[0] - lat)**2 + (x[1] - lon)**2), axis = 1)

## Central park
lat = 40.785091
lon = -73.968285
full_data['distance_to_cp'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:math.sqrt((x[0] - lat)**2 + (x[1] - lon)**2), axis = 1)

In [15]:
## Log-transformation of the price variable
full_data['log_price'] = np.log(full_data['price'])

## Number of total rooms
full_data['rooms'] = full_data['bedrooms'] + full_data['bathrooms'] 

## Number of categorical features listed 
full_data['num_of_features'] = full_data.iloc[:, 7:67].sum(axis = 1)

## Number of half-baths
full_data['half_bathrooms'] = full_data['bathrooms'] - full_data['bathrooms'].apply(int)

## Price per room
full_data['price_per_room'] = full_data[['price','rooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis = 1)

## Price per bedroom
full_data['price_per_bedroom'] = full_data[['price','bedrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis = 1)

## Price per bathroom
full_data['price_per_bathroom'] = full_data[['price','bathrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis = 1)

## Price per feature listed
full_data['price_per_feature'] = full_data[['price','num_of_features']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis = 1)

## Features per room
full_data['features_per_room'] = full_data[['num_of_features','rooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis = 1)

## Features per bedroom
full_data['features_per_bedroom'] = full_data[['num_of_features','bedrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis = 1)

## Features per bathroom
full_data['features_per_bathroom'] = full_data[['num_of_features','bathrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis = 1)

In [16]:
## Making sure address variable is a string
full_data['street_address'] = full_data['street_address'].astype(str)

## Getting a count of observations with same address
street = full_data['street_address'].value_counts()

## Getting a count of observations with same number of bedrooms
bedrooms = full_data['bedrooms'].value_counts()

## Getting a count of observations with same number of bathrooms
bathrooms = full_data['bathrooms'].value_counts()

## Adding count information
full_data['street_count'] = full_data['street_address'].apply(lambda x:street[x] if x == 'nan' else street[x])
full_data['bedrooms_count'] = full_data['bedrooms'].apply(lambda x:bedrooms[x])
full_data['bathrooms_count'] = full_data['bathrooms'].apply(lambda x:bathrooms[x])

In [17]:
## Computing density - number of listings in given area
full_data['pos'] = full_data['longitude'].round(3).astype(str) + '_' + full_data['latitude'].round(3).astype(str)
vals = full_data['pos'].value_counts()
dvals = vals.to_dict()
full_data['density'] = full_data['pos'].apply(lambda x: dvals.get(x, vals.min()))

In [18]:
## Aggregating price variables by street address and computing summary statistics
price_by_address = full_data.groupby('street_address')['price'].agg([np.min, np.max, np.median, np.mean]).reset_index()
price_by_address.columns = ['street_address','min_price_by_address',
                            'max_price_by_address','median_price_by_address','mean_price_by_address']

## Adding aggregated price info
full_data = pd.merge(full_data, price_by_address, how = 'left', on = 'street_address')

## Computing percentile from aggregated price info
full_data['price_percentile_by_address'] = full_data[['price','min_price_by_address','max_price_by_address']]\
.apply(lambda x:(x[0] - x[1]) / (x[2] - x[1]) if (x[2] - x[1]) != 0 else 0.5, axis = 1)

In [19]:
## Aggregating price variables by position and computing summary statistics
price_by_pos = full_data.groupby('pos')['price'].agg([np.min, np.max, np.median, np.mean]).reset_index()
price_by_pos.columns = ['pos','min_price_by_pos',
                            'max_price_by_pos','median_price_by_pos','mean_price_by_pos']

## Adding aggregated price info
full_data = pd.merge(full_data, price_by_pos, how = 'left', on = 'pos')

## Computing percentile from aggregated price info
full_data['price_percentile_by_pos'] = full_data[['price','min_price_by_pos','max_price_by_pos']]\
.apply(lambda x:(x[0] - x[1]) / (x[2] - x[1]) if (x[2] - x[1]) != 0 else 0.5, axis = 1)

In [20]:
## Computing market price for combination of address, bedrooms, and bathrooms
mkt_price = full_data.groupby(['street_address', 'bedrooms', 'bathrooms']).price.mean().reset_index()

## Adding market price information
mkt_price = pd.merge(full_data[['street_address', 'bedrooms', 'bathrooms']], mkt_price, how = 'left', 
                     on = ['street_address', 'bedrooms', 'bathrooms']).price
full_data['mkt_price'] = mkt_price.values

## Computing the difference to market price
full_data['diff_to_mkt_price'] = full_data['price'] - full_data['mkt_price']

## Computing the ratio to market price
full_data['ratio_to_mkt_price'] = full_data['price'] / full_data['mkt_price']

In [21]:
## Condensing redundant variables after feature engineering

laundry_vars = ['Laundry.in.Building', 'Laundry.in.Unit', 'Laundry.In.Building', 'Laundry.In.Unit', 'LAUNDRY', 'Washer.in.Unit', 
                'Dryer.in.Unit', 'Laundry.Room', 'Laundry', 'On.site.laundry', 'On.site.Laundry', 'Washer.Dryer', 'Washer.Dryer.in.building', 
                'In.Unit.Washer.Dryer', 'Washer...Dryer', 'Washer.Dryer.in.Unit']
parking_vars = ['Parking.Space', 'Garage', 'Parking', 'On.site.Garage', 'assigned.parking.space', 'Common.parking.Garage', 'Full.Service.Garage', 
               'On.site.Parking.Lot', 'Private.parking']
valet_vars = ['Valet.Parking', 'Valet']
deck_vars = ['Roof.Deck', 'Balcony', 'Terrace', 'Patio', 'Roof.deck', 'balcony', 'terrace', 'patio', 'private.balcony', 'Private.balcony', 
             'Private.Deck', 'Common.roof.deck', 'ROOFDECK']
outdoor_vars = ['Courtyard', 'Outdoor.Entertainment.Space', 'Private.Outdoor.Space', 'private.outdoor.space', 'Private.outdoor.space', 
                'Common.Outdoor.Space', 'PublicOutdoor', 'Outdoor.Space', 'Outdoor.Areas', 'Common.backyard', 'building.common.outdoor.space']
garden_vars = ['Common.garden', 'garden', 'Garden.Patio', 'Garden', 'Residents.Garden']
dishwasher_vars = ['Dishwasher', 'dishwasher']
gym_vars = ['Fitness.Center', 'Gym.Fitness', 'Health.Club', 'Gym', 'gym', 'Gym.In.Building']
pool_vars = ['Swimming.Pool', 'Pool', 'pool', 'Indoor.Pool']
elevator_vars = ['Elevator', 'elevator']
storage_vars = ['Storage', 'storage', 'Basement.Storage']
internet_vars = ['High.Speed.Internet', 'WiFi', 'WiFi.Access']
bike_vars = ['Bike.room', 'Bike.Room']
pet_friendly_vars = ['Dogs.Allowed', 'Cats.Allowed', 'Pet.Friendly', 'Pets.on.approval']
concierge_vars = ['Concierge', 'Concierge.Service', 'X24.7.Concierge']
doorman_vars = ['Doorman', 'Full.time.doorman', 'Virtual.Doorman', 'FT.Doorman', 'doorman']
super_vars = ['LIVE.IN.SUPER', 'Live.in.superintendent', 'Live.In.Superintendent','Live.in.Super', 'Live.In.Super']
hardwood_vars = ['Hardwood.Floors', 'HARDWOOD', 'Hardwood.floors', 'Hardwood']
ceiling_vars = ['High.ceilings', 'High.Ceilings', 'HIGH.CEILINGS', 'High.Ceiling']
brick_vars = ['EXPOSED.BRICK', 'Exposed.Brick']
construction_vars = ['New.Construction', 'Newly.renovated','Renovated', 'renovated', 'New.construction']
photo_vars = ['Actual.Apt..Photos', 'ACTUAL.APT..PHOTOS']
lounge_vars = ['Residents.Lounge', 'Lounge.room', 'Lounge']
playroom_vars = ['Childrens.Playroom', 'Children.s.Playroom']
ac_vars = ['Central.A.C', 'Air.conditioning']
kitchen_vars = ['EAT.IN.KITCHEN','Eat.In.Kitchen']
no_fee_vars = ['No.Fee', 'NO.FEE']
accessibity_vars = ['Wheelchair.Ramp', 'Wheelchair.Access']
multi_level_vars = ['Multi.Level', 'Multi.level']
fireplace_vars = ['Fireplace', 'Decorative.Fireplace']
highrise_vars = ['Hi.Rise', 'HIGHRISE']
marble_bath_vars = ['Marble.Bath', 'Marble.Bathroom']
prewar_vars = ['Pre.War', 'prewar', 'Prewar']

def condense(data):
    
    ## Condensing data
    data['Has_Laundry'] = np.where(np.sum(data[laundry_vars], axis = 1) > 0, 1, 0)
    data['Has_Parking'] = np.where(np.sum(data[parking_vars], axis = 1) > 0, 1, 0)
    data['Has_Valet'] = np.where(np.sum(data[valet_vars], axis = 1) > 0, 1, 0)
    data['Has_Deck'] = np.where(np.sum(data[deck_vars], axis = 1) > 0, 1, 0)
    data['Has_Outdoor_Area'] = np.where(np.sum(data[outdoor_vars], axis = 1) > 0, 1, 0)
    data['Has_Garden'] = np.where(np.sum(data[garden_vars], axis = 1) > 0, 1, 0)
    data['Has_Dishwasher'] = np.where(np.sum(data[dishwasher_vars], axis = 1) > 0, 1, 0)
    data['Has_Gym'] = np.where(np.sum(data[gym_vars], axis = 1) > 0, 1, 0)
    data['Has_Pool'] = np.where(np.sum(data[pool_vars], axis = 1) > 0, 1, 0)
    data['Has_Elevator'] = np.where(np.sum(data[elevator_vars], axis = 1) > 0, 1, 0)
    data['Has_Storage'] = np.where(np.sum(data[storage_vars], axis = 1) > 0, 1, 0)
    data['Has_Wifi'] = np.where(np.sum(data[internet_vars], axis = 1) > 0, 1, 0)
    data['Has_Bike'] = np.where(np.sum(data[bike_vars], axis = 1) > 0, 1, 0)
    data['Pet_Friendly'] = np.where(np.sum(data[pet_friendly_vars], axis = 1) > 0, 1, 0)
    data['Has_Concierge'] = np.where(np.sum(data[concierge_vars], axis = 1) > 0, 1, 0)
    data['Has_Doorman'] = np.where(np.sum(data[doorman_vars], axis = 1) > 0, 1, 0)
    data['Has_Super'] = np.where(np.sum(data[super_vars], axis = 1) > 0, 1, 0)
    data['Has_Hardwood_Floor'] = np.where(np.sum(data[hardwood_vars], axis = 1) > 0, 1, 0)
    data['Has_High_Ceilings'] = np.where(np.sum(data[ceiling_vars], axis = 1) > 0, 1, 0)
    data['Has_Brick'] = np.where(np.sum(data[brick_vars], axis = 1) > 0, 1, 0)
    data['Has_Renovated'] = np.where(np.sum(data[construction_vars], axis = 1) > 0, 1, 0)
    data['Has_Photos'] = np.where(np.sum(data[photo_vars], axis = 1) > 0, 1, 0)
    data['Has_Lounge'] = np.where(np.sum(data[lounge_vars], axis = 1) > 0, 1, 0)
    data['Has_Playroom'] = np.where(np.sum(data[playroom_vars], axis = 1) > 0, 1, 0)
    data['Has_AC'] = np.where(np.sum(data[ac_vars], axis = 1) > 0, 1, 0)
    data['Has_Kitchen'] = np.where(np.sum(data[kitchen_vars], axis = 1) > 0, 1, 0)
    data['No_Fee'] = np.where(np.sum(data[no_fee_vars], axis = 1) > 0, 1, 0)
    data['Accessible'] = np.where(np.sum(data[accessibity_vars], axis = 1) > 0, 1, 0)
    data['Multi_Level'] = np.where(np.sum(data[multi_level_vars], axis = 1) > 0, 1, 0)
    data['Fire'] = np.where(np.sum(data[fireplace_vars], axis = 1) > 0, 1, 0)
    data['Highrise'] = np.where(np.sum(data[highrise_vars], axis = 1) > 0, 1, 0)
    data['Marble_Bath'] = np.where(np.sum(data[marble_bath_vars], axis = 1) > 0, 1, 0)
    data['Pre_War'] = np.where(np.sum(data[prewar_vars], axis = 1) > 0, 1, 0)
    return data
    
## Applying function to full_data
full_data = condense(full_data)

## Dropping unnecessary columns
to_drop = [laundry_vars, parking_vars, valet_vars, deck_vars, outdoor_vars,garden_vars, dishwasher_vars, gym_vars, pool_vars, elevator_vars, 
           storage_vars, internet_vars, bike_vars, pet_friendly_vars, concierge_vars, doorman_vars, super_vars, hardwood_vars, ceiling_vars, 
           brick_vars, construction_vars, photo_vars, lounge_vars, playroom_vars, ac_vars, kitchen_vars, no_fee_vars, accessibity_vars, 
           multi_level_vars, fireplace_vars, highrise_vars, marble_bath_vars, prewar_vars] 

for cols in to_drop:
    full_data = full_data.drop(columns = cols)

In [26]:
## Label encoding categorical variables
cat_vars = ['Dining.Room', 'Reduced.Fee', 'Exclusive', 'No.pets', 'LOWRISE', 'SIMPLEX', 'Furnished', 'Loft', 'Stainless.Steel.Appliances', 'View', 
            'Green.Building', 'Short.Term.Allowed', 'Subway', 'Granite.Kitchen', 'Light', 'Guarantors.Accepted', 'Skylight', 'Sauna', 'Live.Work', 
            'Duplex', 'Walk.in.Closet.s.', 'Luxury.building', 'Post.War', 'Cable.Satellite.TV', 'Microwave', 'Sublet', 'Shares.OK', 'Has_Laundry', 
            'Has_Parking', 'Has_Valet', 'Has_Deck', 'Has_Outdoor_Area', 'Has_Garden', 'Has_Dishwasher', 'Has_Gym', 'Has_Pool', 'Has_Elevator', 
            'Has_Storage', 'Has_Wifi', 'Has_Bike', 'Pet_Friendly', 'Has_Concierge', 'Has_Doorman', 'Has_Super', 'Has_Hardwood_Floor', 
            'Has_High_Ceilings', 'Has_Brick', 'Has_Renovated', 'Has_Photos', 'Has_Lounge', 'Has_Playroom', 'Has_AC', 'Has_Kitchen', 'No_Fee', 
            'Accessible', 'Multi_Level', 'Fire', 'Highrise', 'Marble_Bath', 'Pre_War']
LE_vars = []
for cat_var in cat_vars:
    LE_var = cat_var
    full_data[LE_var] = LabelEncoder().fit_transform(full_data[cat_var])
    LE_vars.append(LE_var)
    
## OneHot Encoding all catategorical variables
oh_encoder = OneHotEncoder(sparse_output = True).fit(full_data[LE_vars])
oh_sparse = oh_encoder.transform(full_data[LE_vars])

In [27]:
## Splitting full_data back into training and testing sets
training = full_data[full_data['train'] == 1].drop(columns = ['train', 'street_address', 'pos']).reset_index(drop = True)
training['interest_level'] = train['interest_level']

testing = full_data[full_data['train'] == 0].drop(columns = ['train', 'street_address', 'pos']).reset_index(drop = True)
testing['ID'] = test['ID']

In [31]:
## Defining input and target variables
X = training.drop(columns = ['interest_level'])
Y = training['interest_level']

X_test = testing.drop(columns = ['ID'])

In [32]:
## RandomForest:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
                
    ## Building LightGBM model
    model = RandomForestClassifier(n_estimators = 700,
                                   max_depth = 12,
                                   min_samples_split = 5,
                                   min_samples_leaf = 5).fit(X_train, Y_train)   
    
    ## Predicting on X_val and X_test
    model_pred_val = model.predict_proba(X_val)
    model_pred_test = model.predict_proba(X_test)
        
    ## Computing log-loss
    score = log_loss(Y_val, model_pred_val)
    log_loss_scores.append(score)
    preds.append(model_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)

## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
rf_cv_score = np.mean(cv_scores)    
print('Average log-loss of the RandomForest model over 10-folds is:', rf_cv_score)

## Averaging RF model preds
rf_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
rf_preds_test.columns = model.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = rf_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/rf_submission.csv', index = False)

Fold 0 : log-loss-score ==> 0.8642745903808168
Fold 1 : log-loss-score ==> 0.8775092599855586
Fold 2 : log-loss-score ==> 0.8760201258324608
Fold 3 : log-loss-score ==> 0.8487048246308411
Fold 4 : log-loss-score ==> 0.8786928588201087
Fold 5 : log-loss-score ==> 0.8427746524002978
Fold 6 : log-loss-score ==> 0.8844626117665872
Fold 7 : log-loss-score ==> 0.8643928533641023
Fold 8 : log-loss-score ==> 0.878234290227424
Fold 9 : log-loss-score ==> 0.8426762076080894
Average log-loss of the RandomForest model over 10-folds is: 0.8657742275016288


In [ ]:
## XGBoost:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
                
    ## Building LightGBM model
    model = XGBClassifier(n_estimators = 1500,
                          max_depth = 10,
                          learning_rate = 0.17,
                          gamma = 10, 
                          min_child_weight = 0, 
                          subsample = 0.85,
                          colsample_bytree = 0.9, 
                          objective = 'multi:softprob', 
                          eval_metric = 'mlogloss').fit(X_train, Y_train)  
    
    ## Predicting on X_val and X_test
    model_pred_val = model.predict_proba(X_val)
    model_pred_test = model.predict_proba(X_test)
        
    ## Computing log-loss
    score = log_loss(Y_val, model_pred_val)
    log_loss_scores.append(score)
    preds.append(model_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)
        
## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
xgb_cv_score = np.mean(cv_scores)    
print('Average log-loss of the XGBoost model over 10-folds is:', xgb_cv_score)

## Averaging LGBM model preds
xgb_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
xgb_preds_test.columns = model.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = xgb_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/xgb_submission.csv', index = False)

In [36]:
## LightGBM:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
                
    ## Building LightGBM model
    model = LGBMClassifier(n_estimators = 500,
                           max_depth = 4,
                           learning_rate = 0.06,
                           num_leaves = 56,
                           reg_alpha = 2.41, 
                           reg_lambda = 0.15, 
                           subsample = 0.95,
                           colsample_bytree = 0.6).fit(X_train, Y_train)    
    
    ## Predicting on X_val and X_test
    model_pred_val = model.predict_proba(X_val)
    model_pred_test = model.predict_proba(X_test)
        
    ## Computing log-loss
    score = log_loss(Y_val, model_pred_val)
    log_loss_scores.append(score)
    preds.append(model_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)
        
## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
lgbm_cv_score = np.mean(cv_scores)    
print('Average log-loss of the LightGBM model over 10-folds is:', lgbm_cv_score)

## Averaging LGBM model preds
lgbm_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
lgbm_preds_test.columns = model.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = lgbm_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/lgbm_submission.csv', index = False)

Fold 0 : log-loss-score ==> 0.8203649100532943
Fold 1 : log-loss-score ==> 0.8628034085206576
Fold 2 : log-loss-score ==> 0.8407055398653133
Fold 3 : log-loss-score ==> 0.808114477807324
Fold 4 : log-loss-score ==> 0.8478417964727517
Fold 5 : log-loss-score ==> 0.7899799375606462
Fold 6 : log-loss-score ==> 0.8497054161714602
Fold 7 : log-loss-score ==> 0.839633937661767
Fold 8 : log-loss-score ==> 0.8371491948760821
Fold 9 : log-loss-score ==> 0.8162476891534527
Average log-loss of the LightGBM model over 10-folds is: 0.831254630814275


In [38]:
## HistGradientBoosting:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
                
    ## Building LightGBM model
    model = HistGradientBoostingClassifier(max_iter = 500,
                                           max_depth = 9,
                                           learning_rate = 0.01,
                                           l2_regularization = 4.88).fit(X_train, Y_train)
    
    ## Predicting on X_val and X_test
    model_pred_val = model.predict_proba(X_val)
    model_pred_test = model.predict_proba(X_test)
        
    ## Computing log-loss
    score = log_loss(Y_val, model_pred_val)
    log_loss_scores.append(score)
    preds.append(model_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)
        
## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
hist_cv_score = np.mean(cv_scores)    
print('Average log-loss of the HistGB model over 10-folds is:', hist_cv_score)

## Averaging LGBM model preds
hist_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
hist_preds_test.columns = model.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = hist_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/hist_submission.csv', index = False)

Fold 0 : log-loss-score ==> 0.8253009798296974
Fold 1 : log-loss-score ==> 0.8414092444773742
Fold 2 : log-loss-score ==> 0.8289861335771727
Fold 3 : log-loss-score ==> 0.8034629816313835
Fold 4 : log-loss-score ==> 0.8301829335018452
Fold 5 : log-loss-score ==> 0.782276653693204
Fold 6 : log-loss-score ==> 0.8341674063484721
Fold 7 : log-loss-score ==> 0.8230944877909459
Fold 8 : log-loss-score ==> 0.826377898875996
Fold 9 : log-loss-score ==> 0.8065535897179091
Average log-loss of the HistGB model over 10-folds is: 0.8201812309444


In [ ]:
## AdaBoost:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
                
    ## Building LightGBM model
    model = AdaBoostClassifier(n_estimators=500, 
                               random_state=2017, 
                               learning_rate=0.01).fit(X_train, Y_train)
    
    ## Predicting on X_val and X_test
    model_pred_val = model.predict_proba(X_val)
    model_pred_test = model.predict_proba(X_test)
        
    ## Computing log-loss
    score = log_loss(Y_val, model_pred_val)
    log_loss_scores.append(score)
    preds.append(model_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)
        
## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
ada_cv_score = np.mean(cv_scores)    
print('Average log-loss of the AdaBoost model over 10-folds is:', ada_cv_score)

## Averaging LGBM model preds
ada_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
ada_preds_test.columns = model.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = ada_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/ada_submission.csv', index = False)

In [ ]:
## GradientBoosting:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
                
    ## Building LightGBM model
    model = GradientBoostingClassifier(learning_rate=0.04, 
                                       n_estimators=100, 
                                       subsample=0.8, 
                                       random_state=2017,
                                       max_depth=5,
                                       verbose=1).fit(X_train, Y_train)
    
    ## Predicting on X_val and X_test
    model_pred_val = model.predict_proba(X_val)
    model_pred_test = model.predict_proba(X_test)
        
    ## Computing log-loss
    score = log_loss(Y_val, model_pred_val)
    log_loss_scores.append(score)
    preds.append(model_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)
        
## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
gbm_cv_score = np.mean(cv_scores)    
print('Average log-loss of the GBM model over 10-folds is:', gbm_cv_score)

## Averaging LGBM model preds
gbm_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
gbm_preds_test.columns = model.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = gbm_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/gbm_submission.csv', index = False)

In [ ]:
def ada(x_train, y_train, x_valid):
    adaboost = AdaBoostClassifier(n_estimators=50, random_state=2017, learning_rate=0.01)
    ada_train, ada_test = stacking(adaboost, x_train, y_train, x_valid,"ada")
    return ada_train, ada_test,"ada"

def gb(x_train, y_train, x_valid):
    gbdt = GradientBoostingClassifier(learning_rate=0.04, n_estimators=100, subsample=0.8, random_state=2017,max_depth=5,verbose=1)
    gbdt_train, gbdt_test = stacking(gbdt, x_train, y_train, x_valid,"gb")
    return gbdt_train, gbdt_test,"gb"

def et(x_train, y_train, x_valid):
    extratree = ExtraTreesClassifier(n_estimators=1200, max_depth=35, max_features="auto", n_jobs=-1, random_state=2017,verbose=1)
    et_train, et_test = stacking(extratree, x_train, y_train, x_valid,"et")
    return et_train, et_test,"et"

### Ensembling
Averaging predictions from different models using thier optimized hyper-parameter sets

In [ ]:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
    
    ## Initializing val_preds list
    val_preds = list()
                
    ## Building LightGBM model
    model1 = HistGradientBoostingClassifier(max_iter = 500,
                                           max_depth = 9,
                                           learning_rate = 0.01,
                                           l2_regularization = 4.88).fit(X_train, Y_train)   
    
    model2 = LGBMClassifier(n_estimators = 500,
                           max_depth = 3,
                           learning_rate = 0.06,
                           num_leaves = 56,
                           reg_alpha = 2.41, 
                           reg_lambda = 0.15, 
                           subsample = 0.95,
                           colsample_bytree = 0.6).fit(X_train, Y_train)
    
    ## Predicting on X_val and X_test
    model1_pred_val = model1.predict_proba(X_val)
    model2_pred_val = model2.predict_proba(X_val)
    
    model1_pred_test = model1.predict_proba(X_test)
    model2_pred_test = model2.predict_proba(X_test)
    
    ## Averaging val predictions
    val_preds.append(model1_pred_val); val_preds.append(model2_pred_val)
    val_preds = pd.DataFrame(np.mean(val_preds, axis = 0))
    
    ## Computing log-loss
    score = log_loss(Y_val, val_preds)
    log_loss_scores.append(score)
    preds.append(model1_pred_test); preds.append(model2_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)
        
## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
ens_cv_score = np.mean(cv_scores)    
print('Average log-loss of the Ensemble model over 10-folds is:', ens_cv_score)

## Averaging LGBM model preds
ens_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
ens_preds_test.columns = model1.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = ens_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/ens_submission.csv', index = False)

In [ ]:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
    
    ## Initializing val_preds list
    val_preds = list()
                
    ## Building LightGBM model
    model1 = HistGradientBoostingClassifier(max_iter = 500,
                                           max_depth = 9,
                                           learning_rate = 0.01,
                                           l2_regularization = 4.88).fit(X_train, Y_train)   
    
    model2 = LGBMClassifier(n_estimators = 500,
                           max_depth = 3,
                           learning_rate = 0.06,
                           num_leaves = 56,
                           reg_alpha = 2.41, 
                           reg_lambda = 0.15, 
                           subsample = 0.95,
                           colsample_bytree = 0.6).fit(X_train, Y_train)
    
    model3 = XGBClassifier(n_estimators = 1500,
                          max_depth = 10,
                          learning_rate = 0.17,
                          gamma = 10, 
                          min_child_weight = 0, 
                          subsample = 0.85,
                          colsample_bytree = 0.9, 
                          objective = 'multi:softprob', 
                          eval_metric = 'mlogloss').fit(X_train, Y_train) 
    
    ## Predicting on X_val and X_test
    model1_pred_val = model1.predict_proba(X_val)
    model2_pred_val = model2.predict_proba(X_val)
    model3_pred_val = model3.predict_proba(X_val)
    
    model1_pred_test = model1.predict_proba(X_test)
    model2_pred_test = model2.predict_proba(X_test)
    model3_pred_test = model3.predict_proba(X_test)
    
    ## Averaging val predictions
    val_preds.append(model1_pred_val); val_preds.append(model2_pred_val); val_preds.append(model3_pred_val)
    val_preds = pd.DataFrame(np.mean(val_preds, axis = 0))
    
    ## Computing log-loss
    score = log_loss(Y_val, val_preds)
    log_loss_scores.append(score)
    preds.append(model1_pred_test); preds.append(model2_pred_test); preds.append(model3_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)
        
## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
ens_cv_score = np.mean(cv_scores)    
print('Average log-loss of the Ensemble model over 10-folds is:', ens_cv_score)

## Averaging LGBM model preds
ens_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
ens_preds_test.columns = model1.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = ens_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/ens2_submission.csv', index = False)

In [ ]:
cv_scores, log_loss_scores = list(), list()
preds = list()

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):

    ## Splitting the data 
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
    
    ## Initializing val_preds list
    val_preds = list()
                
    ## Building LightGBM model
    model1 = HistGradientBoostingClassifier(max_iter = 300,
                                           max_depth = 12,
                                           learning_rate = 0.02,
                                           l2_regularization = 4.26).fit(X_train, Y_train)    
    
    model2 = LGBMClassifier(n_estimators = 400,
                           max_depth = 3,
                           learning_rate = 0.27,
                           num_leaves = 56,
                           reg_alpha = 9.96, 
                           reg_lambda = 0.85, 
                           subsample = 0.85,
                           colsample_bytree = 0.8).fit(X_train, Y_train) 
    
    model4 = RandomForestClassifier(n_estimators = 800,
                                   max_depth = 11,
                                   min_samples_split = 50,
                                   min_samples_leaf = 5).fit(X_train, Y_train)  
    
    ## Predicting on X_val and X_test
    model1_pred_val = model1.predict_proba(X_val)
    model2_pred_val = model2.predict_proba(X_val)
    #model3_pred_val = model3.predict_proba(X_val)
    model4_pred_val = model4.predict_proba(X_val)
    
    model1_pred_test = model1.predict_proba(X_test)
    model2_pred_test = model2.predict_proba(X_test)
    #model3_pred_test = model3.predict_proba(X_test)
    model4_pred_test = model3.predict_proba(X_test)
    
    ## Averaging val predictions
    val_preds.append(model1_pred_val); val_preds.append(model2_pred_val); val_preds.append(model4_pred_val)
    val_preds = pd.DataFrame(np.mean(val_preds, axis = 0))
    
    ## Computing log-loss
    score = log_loss(Y_val, val_preds)
    log_loss_scores.append(score)
    preds.append(model1_pred_test); preds.append(model2_pred_test); preds.append(model4_pred_test)
    print('Fold', i, ': log-loss-score ==>', score)
        
## Appending average cv scores
cv_scores.append(np.mean(log_loss_scores))
ens_cv_score = np.mean(cv_scores)    
print('Average log-loss of the Ensemble model over 10-folds is:', ens_cv_score)

## Averaging LGBM model preds
ens_preds_test = pd.DataFrame(np.mean(preds, axis = 0))

## Renaming columns by label class
ens_preds_test.columns = model1.classes_

## Creating submission file
sub[['high', 'medium', 'low']] = ens_preds_test[[2, 1, 0]]
sub.to_csv('Submissions/ens3_submission.csv', index = False)